In [92]:
import torch

order = 4
H = 23
W = 23
in_channels = 16
out_channels = 32
kernel_size = 5


In [93]:
from groups.discrete_so2 import DiscreteSO2
from models.layers.group_conv import GroupConv

group = DiscreteSO2(order=4, device="cpu")
conv = GroupConv(
    group=group,
    in_channels=in_channels,
    out_channels=out_channels,
    kernel_size=kernel_size
)


Check equivariance

In [94]:
group_in_signal = torch.randn(in_channels, order, H, W)
batched_group_signal = group_in_signal.view(1, *group_in_signal.shape)

for g in group.elements():
  rotated_group_in_signal = group.left_action_on_group_signal(
      g.view(1, 1), group_in_signal).squeeze(0)
  rotate_then_conv = conv(
      rotated_group_in_signal.view(1, *group_in_signal.shape))

  convolved_in_signal = conv(group_in_signal.view(
      1, *group_in_signal.shape)).squeeze(0)
  conv_then_rotated = group.left_action_on_group_signal(
      g.view(1, 1), convolved_in_signal)

  # check equivariance
  print(
      torch.allclose(conv_then_rotated,
                     rotate_then_conv, atol=1e-5, rtol=1e-5)
  )


True
True
True
True
